# LDA

#### LDA Details
- Topic Modelling
    - topic = a repeating pattern of co-occurring terms in a corpus
- Parameters of LDA
    - Alpha = document <-> topic density
    - Beta = topic <-> word density
    - Higher alpha = more topics
    - High beta = larger number of words
- LDA Steps
    1. Prepare documents (put in list)
    2. Clean & Preprocess
        - remove punctuation and stopwords
        - convert word to root word (running and ran would be both converted to run)
    3. Prepare Document-Term Matrix
        - create dictionary from corpus (literally use gensim dictionary, it is the easiest way)
    4. Run LDA Model
        - create object for LDA and train on document-term matrix
        - fun fact-- gensim has an open-source LDA model that we can use!
    5. Get results!
        - To improve results
            - Frequency Filter (filter out words used less than 3 times and the top 10-20% of words)
            - Parts of Speech tag filter (in our case I think this would be a bad thing)
            - Base-wise LDA (we can look into this later)

In [9]:
# Imports that will be necessary

import re
import numpy as np
import pandas as pd
from pprint import pprint
import json

# ML imports
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy

# for lemmatization -- but can't we use NLTK lemmatizer?
from nltk.stem import WordNetLemmatizer

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [3]:
# Preprocess Data

subreddits = ['depression', 'Anxiety', 'foreveralone', 'socialanxiety', 'SuicideWatch', 
                    'berkeley', 'PowerLedger', 'TalesFromYourServer', 'tifu']
total = []
temp = []
for subreddit in subreddits:
    with open('datasets/'+subreddit+'_text_samples_extended.json') as f:
        print('opening' + 'datasets/'+subreddit+'_text_samples_extended.json')
        data = json.load(f)
    
    for x in range(0, len(data['data'])):
        if x % 100 == 0:
            print(x)
        if 'selftext' in data['data'][x]:
            # filter out newline at a later time
            total.append(gensim.utils.simple_preprocess(data.get('data')[x].get('selftext')))            

openingdatasets/depression_text_samples_extended.json
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
openingdatasets/Anxiety_text_samples_extended.json
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300

In [14]:
print(total[:1])

[['cant', 'keep', 'the', 'thought', 'of', 'potential', 'rape', 'out', 'of', 'my', 'head', 'every', 'single', 'day', 'know', 'men', 'get', 'raped', 'too', 'but', 'women', 'have', 'higher', 'chance', 'of', 'it', 'happening', 'to', 'them', 'men', 'in', 'general', 'dont', 'go', 'around', 'living', 'in', 'fear', 'of', 'being', 'raped', 'or', 'being', 'physically', 'vulnerable', 'when', 'they', 'step', 'outside', 'their', 'house', 'know', 'rape', 'isnt', 'womans', 'fault', 'and', 'rapist', 'are', 'just', 'gonna', 'be', 'rapist', 'but', 'just', 'cant', 'help', 'the', 'fact', 'that', 'rather', 'just', 'not', 'be', 'born', 'girl', 'to', 'even', 'have', 'to', 'deal', 'with', 'this', 'thought', 'of', 'rape', 'and', 'worrying', 'about', 'rape', 'everyday', 'and', 'getting', 'pregnant', 'men', 'dont', 'ever', 'worry', 'about', 'getting', 'pregnant', 'at', 'all', 'no', 'matter', 'what', 'hate', 'that', 'have', 'to', 'take', 'birth', 'control', 'if', 'rape', 'were', 'to', 'happen', 'or', 'to', 'just'

In [5]:
# bigram and trigram model
bigram = gensim.models.Phrases(total, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[total], threshold=100)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[total[0]]])

C:\Users\prang\Anaconda3\lib\site-packages\gensim\models\phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['cant', 'keep', 'the', 'thought', 'of', 'potential', 'rape', 'out', 'of', 'my', 'head', 'every', 'single', 'day', 'know', 'men', 'get', 'raped', 'too', 'but', 'women', 'have', 'higher', 'chance', 'of', 'it', 'happening', 'to', 'them', 'men', 'in', 'general', 'dont', 'go', 'around', 'living', 'in', 'fear', 'of', 'being', 'raped', 'or', 'being', 'physically', 'vulnerable', 'when', 'they', 'step', 'outside', 'their', 'house', 'know', 'rape_isnt_womans', 'fault', 'and', 'rapist', 'are', 'just', 'gonna', 'be', 'rapist', 'but', 'just', 'cant', 'help', 'the', 'fact', 'that', 'rather', 'just', 'not', 'be', 'born', 'girl', 'to', 'even', 'have', 'to', 'deal', 'with', 'this', 'thought', 'of', 'rape', 'and', 'worrying', 'about', 'rape', 'everyday', 'and', 'getting', 'pregnant', 'men', 'dont', 'ever', 'worry', 'about', 'getting', 'pregnant', 'at', 'all', 'no', 'matter', 'what', 'hate', 'that', 'have', 'to', 'take', 'birth', 'control', 'if', 'rape', 'were', 'to', 'happen', 'or', 'to', 'just', 'be',

In [6]:
# get rid of stopwords, make bigrams and trigrams from our data

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
data_words_nostops = remove_stopwords(total)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

In [ ]:
id2word[0]

In [ ]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]